In [ ]:
# Setup and Imports

import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

print("🚀 CLIMATE DATASET UNIFICATION - JUPYTER VERSION")
print("="*60)
print("Combining all validated datasets (92.1% validation score)")
print("="*60)

In [ ]:
# Load All Datasets

def load_all_datasets():
    """Load all validated climate datasets"""
    
    print("📁 LOADING ALL CLIMATE DATASETS")
    print("="*50)
    
    datasets = {}
    
    # Define all dataset files
    dataset_files = {
        'ch4_concentration': 'Ch4_Concentration_Ppm_TimeSeries_2010-2024_ML_READY.csv',
        'temperature': 'Temperature_TimeSeries_2010-2024_ML_READY.csv',
        'precipitation': 'Precipitation_TimeSeries_2010-2024_ML_READY.csv',
        'soil_moisture': 'Soil_Moisture_TimeSeries_2010-2024_ML_READY.csv',
        'elevation': 'Elevation_TimeSeries_2010-2024_ML_READY.csv',
        'permafrost_zones': 'Permafrost_Zones_TimeSeries_2010-2024_ML_READY.csv',
        'permafrost_extent': 'Permafrost_Extent_TimeSeries_2010-2024_ML_READY.csv',
        'wetlands': 'Wetland_Fraction_TimeSeries_2010-2024_ML_READY.csv',
        'industrial_emissions': 'Ch4_Emissions_TimeSeries_2010-2024_ML_READY.csv'
    }
    
    # Load each dataset
    for name, filename in dataset_files.items():
        try:
            df = pd.read_csv(filename)
            datasets[name] = df
            print(f"  ✅ {name}: {len(df):,} records, {df['pixel_id'].nunique():,} pixels")
        except FileNotFoundError:
            print(f"  ❌ {name}: File not found - {filename}")
        except Exception as e:
            print(f"  ⚠️  {name}: Error loading - {e}")
    
    print(f"\n📊 Successfully loaded {len(datasets)} datasets")
    return datasets

# Load all datasets
datasets = load_all_datasets()

In [ ]:
# Find Spatial Intersection

def find_spatial_intersection(datasets):
    """Find common pixels across all datasets"""
    
    print("\n🗺️  FINDING SPATIAL INTERSECTION")
    print("="*50)
    
    # Get pixel sets for each dataset
    pixel_sets = {}
    for name, df in datasets.items():
        pixels = set(df['pixel_id'].unique())
        pixel_sets[name] = pixels
        print(f"  {name}: {len(pixels):,} unique pixels")
    
    # Find intersection of all pixel sets
    all_pixels = list(pixel_sets.values())
    common_pixels = set.intersection(*all_pixels)
    
    print(f"\n🎯 INTERSECTION RESULTS:")
    print(f"  Common pixels across all datasets: {len(common_pixels):,}")
    
    # Calculate coverage for each dataset
    print(f"\n📈 Coverage analysis:")
    for name, pixels in pixel_sets.items():
        coverage = (len(common_pixels) / len(pixels)) * 100
        print(f"  {name}: {coverage:.1f}% of pixels retained")
    
    return common_pixels

# Find common pixels
common_pixels = find_spatial_intersection(datasets)

In [ ]:
# Create Unified Dataset

def create_unified_dataset(datasets, common_pixels):
    """Create one unified dataset with all variables"""
    
    print(f"\n🔄 CREATING UNIFIED DATASET")
    print("="*50)
    
    # Start with CH4 as base (target variable)
    base_dataset = 'ch4_concentration'
    print(f"Using {base_dataset} as base structure...")
    
    # Filter base dataset to common pixels
    unified_df = datasets[base_dataset][
        datasets[base_dataset]['pixel_id'].isin(common_pixels)
    ].copy()
    
    print(f"  Base dataset filtered: {len(unified_df):,} records")
    
    # Merge each additional dataset
    merge_order = [name for name in datasets.keys() if name != base_dataset]
    
    for dataset_name in merge_order:
        print(f"  Merging {dataset_name}...")
        
        # Filter to common pixels
        merge_df = datasets[dataset_name][
            datasets[dataset_name]['pixel_id'].isin(common_pixels)
        ].copy()
        
        # Get data columns (exclude coordinate/time columns)
        data_cols = [col for col in merge_df.columns 
                    if col not in ['pixel_id', 'latitude', 'longitude', 'year']]
        
        # Prepare merge columns
        merge_cols = ['pixel_id', 'year'] + data_cols
        merge_subset = merge_df[merge_cols]
        
        # Merge with unified dataset
        before_merge = len(unified_df)
        unified_df = unified_df.merge(
            merge_subset, 
            on=['pixel_id', 'year'], 
            how='inner'
        )
        after_merge = len(unified_df)
        
        print(f"    Records: {before_merge:,} → {after_merge:,}")
        
        if after_merge < before_merge:
            print(f"    ⚠️  Lost {before_merge - after_merge:,} records in merge")
    
    return unified_df

# Create the unified dataset
unified_df = create_unified_dataset(datasets, common_pixels)

In [ ]:
# Analyze Unified Dataset

def analyze_unified_dataset(unified_df):
    """Analyze the structure and quality of unified dataset"""
    
    print(f"\n📊 UNIFIED DATASET ANALYSIS")
    print("="*50)
    
    print(f"📈 Dataset dimensions:")
    print(f"  Total records: {len(unified_df):,}")
    print(f"  Unique pixels: {unified_df['pixel_id'].nunique():,}")
    print(f"  Years covered: {unified_df['year'].min()}-{unified_df['year'].max()}")
    print(f"  Total variables: {len(unified_df.columns)}")
    
    print(f"\n🗂️  Variable inventory:")
    data_columns = [col for col in unified_df.columns 
                   if col not in ['pixel_id', 'latitude', 'longitude', 'year']]
    
    for i, col in enumerate(data_columns, 1):
        col_stats = unified_df[col].describe()
        missing_pct = (unified_df[col].isnull().sum() / len(unified_df)) * 100
        print(f"  {i:2d}. {col}: {col_stats['min']:.3f} to {col_stats['max']:.3f} "
              f"(missing: {missing_pct:.1f}%)")
    
    # Spatial coverage
    print(f"\n🗺️  Spatial coverage:")
    lat_range = f"{unified_df['latitude'].min():.2f}° to {unified_df['latitude'].max():.2f}°N"
    lon_range = f"{unified_df['longitude'].min():.2f}° to {unified_df['longitude'].max():.2f}°W"
    print(f"  Latitude: {lat_range}")
    print(f"  Longitude: {lon_range}")
    
    # Temporal coverage
    print(f"\n📅 Temporal coverage:")
    years = sorted(unified_df['year'].unique())
    year_counts = unified_df['year'].value_counts().sort_index()
    print(f"  Years: {years[0]}-{years[-1]} ({len(years)} years)")
    print(f"  Records per year: {year_counts.min():,} to {year_counts.max():,}")
    
    # Data completeness
    print(f"\n✅ Data quality:")
    total_cells = len(unified_df) * len(data_columns)
    missing_cells = unified_df[data_columns].isnull().sum().sum()
    completeness = ((total_cells - missing_cells) / total_cells) * 100
    print(f"  Overall completeness: {completeness:.2f}%")
    print(f"  Missing values: {missing_cells:,} out of {total_cells:,} cells")
    
    return data_columns

# Analyze the unified dataset
data_columns = analyze_unified_dataset(unified_df)

In [ ]:
# Prepare ML-Ready Format

def prepare_ml_ready_format(unified_df, data_columns):
    """Prepare the dataset in optimal format for machine learning"""
    
    print(f"\n🤖 PREPARING ML-READY FORMAT")
    print("="*50)
    
    # Create ML version with proper ordering
    ml_df = unified_df.copy()
    
    # Reorder columns for ML convenience
    # Target variable first (CH4 concentration)
    target_col = 'ch4_concentration_ppm'
    feature_cols = [col for col in data_columns if col != target_col]
    
    # Standard column order: pixel_id, lat, lon, year, target, features
    column_order = ['pixel_id', 'latitude', 'longitude', 'year', target_col] + feature_cols
    ml_df = ml_df[column_order]
    
    print(f"✅ Column organization:")
    print(f"  Target variable: {target_col}")
    print(f"  Feature variables: {len(feature_cols)}")
    print(f"  Total columns: {len(column_order)}")
    
    # Display column order
    print(f"\n📋 Final column order:")
    for i, col in enumerate(column_order, 1):
        marker = "🎯" if col == target_col else "📊" if col in feature_cols else "📍"
        print(f"  {i:2d}. {marker} {col}")
    
    return ml_df

# Prepare ML format
ml_ready_df = prepare_ml_ready_format(unified_df, data_columns)

In [ ]:
# Save Unified Dataset

def save_unified_dataset(unified_df, data_columns):
    """Save the unified dataset in multiple formats"""
    
    print(f"\n💾 SAVING UNIFIED DATASET")
    print("="*50)
    
    # Main unified dataset
    main_file = 'Unified_Climate_Dataset_2010-2024_ML_READY.csv'
    unified_df.to_csv(main_file, index=False)
    file_size = os.path.getsize(main_file) / (1024*1024)  # MB
    print(f"  ✅ Main dataset: {main_file}")
    print(f"     Size: {len(unified_df):,} records × {len(unified_df.columns)} variables ({file_size:.1f} MB)")
    
    # Create feature matrix (one row per pixel with time-averaged features)
    print(f"\n📊 Creating feature matrix (pixel-level averages)...")
    feature_matrix = unified_df.groupby(['pixel_id', 'latitude', 'longitude'])[data_columns].mean().reset_index()
    
    feature_file = 'Climate_Feature_Matrix_2010-2024_Averaged.csv'
    feature_matrix.to_csv(feature_file, index=False)
    feature_size = os.path.getsize(feature_file) / (1024*1024)  # MB
    print(f"  ✅ Feature matrix: {feature_file}")
    print(f"     Size: {len(feature_matrix):,} pixels × {len(feature_matrix.columns)} variables ({feature_size:.1f} MB)")
    
    # Create summary report
    summary_file = 'Unified_Dataset_Summary_Report.txt'
    with open(summary_file, 'w') as f:
        f.write("UNIFIED CLIMATE DATASET SUMMARY REPORT\n")
        f.write("="*50 + "\n\n")
        f.write(f"Creation Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Validation Score: 92.1% (Policy-Ready)\n\n")
        f.write(f"DATASET STRUCTURE:\n")
        f.write(f"- Total Records: {len(unified_df):,}\n")
        f.write(f"- Unique Pixels: {unified_df['pixel_id'].nunique():,}\n")
        f.write(f"- Time Period: {unified_df['year'].min()}-{unified_df['year'].max()}\n")
        f.write(f"- Variables: {len(data_columns)}\n\n")
        f.write(f"VARIABLES INCLUDED:\n")
        for i, col in enumerate(data_columns, 1):
            f.write(f"{i:2d}. {col}\n")
        f.write(f"\nSPATIAL COVERAGE:\n")
        f.write(f"- Latitude: {unified_df['latitude'].min():.2f}° to {unified_df['latitude'].max():.2f}°N\n")
        f.write(f"- Longitude: {unified_df['longitude'].min():.2f}° to {unified_df['longitude'].max():.2f}°W\n")
        f.write(f"\nDATA QUALITY:\n")
        total_cells = len(unified_df) * len(data_columns)
        missing_cells = unified_df[data_columns].isnull().sum().sum()
        completeness = ((total_cells - missing_cells) / total_cells) * 100
        f.write(f"- Completeness: {completeness:.2f}%\n")
        f.write(f"- Missing Values: {missing_cells:,}\n")
    
    print(f"  ✅ Summary report: {summary_file}")
    
    return main_file, feature_file

# Save all formats
main_file, feature_file = save_unified_dataset(ml_ready_df, data_columns)

In [ ]:
# Final Summary

# Final success summary
print(f"\n" + "="*60)
print(f"🎉 UNIFICATION COMPLETE!")
print(f"="*60)
print(f"✅ Unified dataset created: {len(ml_ready_df):,} records")
print(f"✅ Spatial coverage: {ml_ready_df['pixel_id'].nunique():,} pixels")
print(f"✅ Variables combined: {len(data_columns)}")
print(f"✅ Time period: {ml_ready_df['year'].min()}-{ml_ready_df['year'].max()}")
print(f"✅ Validation score maintained: 92.1% (POLICY-READY)")

print(f"\n📁 Files created:")
print(f"  1. {main_file} - Complete time series dataset")
print(f"  2. {feature_file} - Pixel-averaged feature matrix") 
print(f"  3. Unified_Dataset_Summary_Report.txt - Documentation")

print(f"\n🎯 Ready for:")
print(f"   • Machine learning model development")
print(f"   • Policy analysis and visualization")
print(f"   • Academic research and publication")
print(f"   • Government decision support")



In [ ]:
# Quick preview of unified dataset
print(f"\n👀 DATASET PREVIEW:")
print(ml_ready_df.head())


print(f"\n📊 VARIABLE SUMMARY:")
print(ml_ready_df.describe())

In [ ]:
import pandas as pd

# Load your DataFrame
df = pd.read_csv("Unified_Climate_Dataset_2010-2024_ML_READY.csv")  # Replace with your actual file

# Move 'ch4_concentration_ppm' to the end
target = 'ch4_concentration_ppm'
columns = [col for col in df.columns if col != target] + [target]
df = df[columns]

# Save the rearranged DataFrame (optional)
df.to_csv("Unified_Climate_Dataset_2010-2024_ML_READY_2.csv", index=False)


In [ ]:
Modeling Pipeline
The harmonized dataset is what you then plan to feed into ML models (LSTM, XGBoost, CNNs) to predict methane hotspots by 2030.

AR(2) forecasting filled in missing years (2023–2024) for dynamic features.

Static features (land cover, elevation) were replicated across years.